In [ ]:
import sleap

import matplotlib.pyplot as plt
import cv2

import scipy

from pathlib import Path

import numpy as np

# Data definition

In [ ]:
#labels = sleap.Labels.load_file("/Users/ulric/Documents/Documents/MaximeModel/LabeledFrames.train.pkg.slp")
labels = sleap.Labels.load_file("/mnt/labserver/DURRIEU_Matthias/Code/Sleap_Pretrained_Maxime/LabeledFrames.train.pkg.slp")

In [ ]:
# Get first frame

frame0 = labels[0]

In [ ]:
# Get instances

instances = frame0.instances

instances

In [ ]:
pts = instances[0].points

In [ ]:
array = instances[0].numpy()

array

In [ ]:
# Add a new value to the array

array = np.append(array, [0,0])

In [ ]:
instances[0].pts = sleap.Instance.from_numpy(array, skeleton=labels.skeletons[0])


In [ ]:
array

In [ ]:
instances[5].points[0].y

In [ ]:
frame1 = labels[1]
instances1 = frame1.instances
instances1

In [ ]:
# Sort instances by y position
instances1.sort(key=lambda x: x[0].y)
instances1
#instances.sort(key=lambda x: x.centroid[1])

In [ ]:
labels[1].instances

Notes : 1) It's fairly easy to sort flies instances by their y positions 2) because the subvariables (ex: instances) are symlinked to the original label variable, the orginial can be edited by editing the subvariable.

In [ ]:
len(labels)

In [ ]:
for f in range(len(labels)):
    labels[f].instances.sort(key=lambda x: x[0].y)

In [ ]:
labels[10].plot()

In [ ]:
frame_0 = labels[0]

frame_0

In [ ]:
img = frame_0.image

img.shape

In [ ]:
# Reshape img to a 832x832 image
img_shaped = img[0:832, 0:832]

plt.imshow(img_shaped, cmap='gray')

# Comparison with cropped video

In [ ]:
CropPath = '/Users/ulric/Movies/TrainingVids/Crops/Arena1/Arena1.mp4'
# Display the frame 6254 of the video CropPath
cap = cv2.VideoCapture(CropPath)
cap.set(1, 6254)
ret, frame = cap.read()
plt.imshow(frame)


Looking good, do the same with arena 4 which is slightly more distinguishable

In [ ]:
CropPath4 = '/Users/ulric/Movies/TrainingVids/Crops/Arena4/Arena4.mp4'
# Display the frame 6254 of the video CropPath
cap = cv2.VideoCapture(CropPath4)
cap.set(1, 6254)
ret, frame = cap.read()
plt.imshow(frame)

Below I tested shifting by 6 frames, let's see if this is consistent

In [ ]:
cap = cv2.VideoCapture(CropPath4)
cap.set(1, 6254-6)
ret, frame = cap.read()
plt.imshow(frame)

Seems to be! Then shift everything by 6 frames should be simpler than working on subsets of the videos.

 Let's check the arena 5

In [ ]:
CropPath5 = '/Users/ulric/Movies/TrainingVids/Crops/Arena5/Arena5.mp4'
# Display the frame 6254 of the video CropPath
cap = cv2.VideoCapture(CropPath5)
cap.set(1, 6254)
ret, frame = cap.read()
plt.imshow(frame)

Might be some inconsistency during the writing process. Then better strategy is to use the cropping parameters and generate the cropped frames from the original labeled frames.

In [ ]:
frame_1 = labels[1]

frame_1

img = frame_1.image

img_shaped = img[0:832, 0:832]

plt.imshow(img_shaped, cmap='gray')

In [ ]:
CropPath = '/Users/ulric/Movies/TrainingVids/Crops/Arena1/Arena1.mp4'
# Display the frame corresponding to the frame_1 idx of the video CropPath
cap = cv2.VideoCapture(CropPath)
cap.set(1, frame_1.frame_idx-6)
ret, frame = cap.read()
plt.imshow(frame)


There are 6 frames discrepancy between the original and the cropped frames. This is due to the fact that the cropping is done on the original frames, while the labeled frames are generated from the cropped frames. This is not a problem, as long as there is the same discrepancy everywhere.

Let's check with some other frame closer to the beginning of the video

In [ ]:
# List frames indices from the labels
frame_idx = []
for i in range(len(labels)):
    frame_idx.append(labels[i].frame_idx)
    
frame_idx

In [ ]:
min(frame_idx)

In [ ]:
frame_14 = labels[14]

frame_14

img = frame_14.image

img_shaped = img[0:832, 0:832]

plt.imshow(img_shaped, cmap='gray')

In [ ]:
CropPath = '/Users/ulric/Movies/TrainingVids/Crops/Arena1/Arena1.mp4'
# Display the frame corresponding to the frame_1 idx of the video CropPath
cap = cv2.VideoCapture(CropPath)
cap.set(1, frame_14.frame_idx-6)
ret, frame = cap.read()
plt.imshow(frame)

In [ ]:
CropPath3 = '/Users/ulric/Movies/TrainingVids/Crops/Arena3/Arena3.mp4'
# Display the frame corresponding to the frame_1 idx of the video CropPath
cap = cv2.VideoCapture(CropPath3)
cap.set(1, frame_14.frame_idx-6)
ret, frame = cap.read()
plt.imshow(frame)

Ffmpeg can screw up when trimming if I forget a flag to synchronise frames. Most of the time this will happen on the first frames. OpenCV should not have this problem, which explains why the video writing was fine.

# Add labels to cropped frames

First work with one frame on one video to make sure the syntax is right.

In [ ]:
#param_path = '/Users/ulric/Movies/TrainingVids/Arena_indices.npy'
param_path = '/mnt/labserver/DURRIEU_Matthias/Code/Sleap_Pretrained_Maxime/TrainingVideo/Arena_indices.npy'
# Load the parameters
Arena_indices = np.load(param_path, allow_pickle=True)

# Convert it to a list
Arena_indices = Arena_indices.tolist()

Arena_indices


Generate a test frame

In [ ]:
CropPath = '/Users/ulric/Movies/TrainingVids/Crops/Arena1/Arena1.mp4'
# Display the frame corresponding to the frame_1 idx of the video CropPath
cap = cv2.VideoCapture(CropPath)
cap.set(1, frame_1.frame_idx-6)
ret, frame = cap.read()
plt.imshow(frame)


In [ ]:
# get the labels from the original frame
frame_1 = labels[1]

labs = frame_1.instances[0].points

labs


In [ ]:
CorrLabs = labs

for i in range(len(labs)):
    CorrLabs[i].y = labs[i].y - Arena_indices[0][0]
    
        


In [ ]:
pts = frame_1.instances[0].numpy()

In [ ]:
LabFrame = sleap.LabeledFrame(video = sleap.load_video(CropPath), frame_idx = frame_1.frame_idx-6, instances = [sleap.Instance.from_numpy(pts, skeleton=labels.skeletons[0])])


LabFrame

First converting the instances points as numpy array worked well to be able to associate frame with instances. Now I need to convert them using sleap methods and skeleton definition to have the edges too

In [ ]:
LabFrame.instances

In [ ]:
sleap.nn.viz.plot_instance(LabFrame.instances[0])

In [ ]:
LabFrame.plot()

In [ ]:
labels[1].instances

In [ ]:
labels[1].plot()

### Important note: the labels are not sorted by value but by the order they were added to the frame. Frames need to be sorted by y value before processing.

# Repeat the same procedure with all the crops of the same frame

In [ ]:
# Reload the original labels for a fresh start

labels = sleap.Labels.load_file("/mnt/labserver/DURRIEU_Matthias/Code/Sleap_Pretrained_Maxime/LabeledFrames.train.pkg.slp")
#labels = sleap.Labels.load_file("/Users/ulric/Documents/Documents/MaximeModel/LabeledFrames.train.pkg.slp")

for f in range(len(labels)):
    labels[f].instances.sort(key=lambda x: x[0].y)

# First define which frame it is : same as the original frame - 6


frame_idx = labels[1].frame_idx - 6

# Create the dataset

CropLabsData = []

#LabFrame = sleap.io.dataset.LabeledFrameDataset(labels, video_paths = [CropPath], frame_indices = [frame_idx])

In [ ]:
#param_path = '/Users/ulric/Movies/TrainingVids/Arena_indices.npy'
param_path = '/mnt/labserver/DURRIEU_Matthias/Code/Sleap_Pretrained_Maxime/TrainingVideo/Arena_indices.npy'
# Load the parameters
Arena_indices = np.load(param_path, allow_pickle=True)

# Convert it to a list
Arena_indices = Arena_indices.tolist()

Arena_indices


In [ ]:
# get all first values of elements in Arena_indices
Crops_adjust = [x[0] for x in Arena_indices]
Crops_adjust



In [ ]:
# Get the path to all the videos in the /Users/ulric/Movies/TrainingVids/Crops folder
PathToCrops = Path("/mnt/labserver/DURRIEU_Matthias/Code/Sleap_Pretrained_Maxime/TrainingVideo/Crops/")
VidPaths = PathToCrops.rglob("*.mp4")
# Put the path to the videos in a list
CropVids = [vid.as_posix() for vid in VidPaths]
# Sort the list
CropVids.sort()
CropVids

Below I tried embedding this in a dataset, but video referencing was bad. Instead I'll first create the dataset then fill it with labeled frames.

In [ ]:
vids = [sleap.load_video(v) for v in CropVids]
vids

Training_data_Cropped = sleap.io.dataset.Labels(
    labeled_frames = [],
    videos = vids,
    skeletons = labels.skeletons,
)

Training_data_Cropped

In [ ]:
Training_data_Cropped.videos[1]

In [ ]:
# This chunk of code is to adjust the labels to the cropped videos : For each labeled frame, for each instance in this labeled frame, for each point in these instances, adjust the y coordinate by the corresponding value in Crops_adjust
for f in range(len(labels)):
    frame = labels[f]
    for i in range(len(frame.instances)):
        inst = frame.instances[i]
        for j in range(len(inst.points)):
            inst.points[j].y = inst.points[j].y - Crops_adjust[i]
        pts = inst.numpy()
        Training_data_Cropped.labeled_frames.append(sleap.LabeledFrame(
        video = Training_data_Cropped.videos[i], 
        frame_idx = labels[f].frame_idx-6, 
        instances = [sleap.Instance.from_numpy(pts, skeleton=labels.skeletons[0])]))
            
            


In [ ]:
Training_data_Cropped

In [ ]:
Training_data_Cropped[0].plot()

Weird, the labels seem to be shifted as if they were already good but by adjusting them I messed them up.

*warning* : Always reload original dataset otherwise adjustments will be kept from previous runs

In [ ]:
CropLabsData[258].plot()

## Bonus : make it a proper dataset

Right now it is a list of individual labeled frames, there must be a better way to implement this

In [ ]:
labels

In [ ]:
labels.video

In [ ]:
Training_data_Cropped = sleap.io.dataset.Labels(
    #labeled_frames = CropLabsData,
    videos = vids,
    skeletons = labels.skeletons,
)

Training_data_Cropped

In [ ]:
CropLabsData

I just need to embed this in a Dataset class and it should be good to go

In [ ]:
vids = [sleap.load_video(v) for v in CropVids]
vids

In [ ]:
Training_data_Cropped = sleap.io.dataset.Labels(
    labeled_frames = CropLabsData,
    #videos = vids,
    skeletons = labels.skeletons,
)

Training_data_Cropped

In [ ]:
Training_data_Cropped.videos

In [ ]:
Training_data_Cropped.videos = vids

Training_data_Cropped.videos

In [ ]:
# Save the dataset

Training_data_Cropped.save("/mnt/labserver/DURRIEU_Matthias/Code/Training_data_Cropped.train.pkg.slp")

#sleap.save(Training_data_Cropped, "/Volumes/Ramdya-Lab/DURRIEU_Matthias/Code/Training_data_Cropped.train.pkg.slp")

# How to get the different instances

In [ ]:

LabsFly1 = frame_0.instances[0]

LabsFly1

## How to edit labels in an instance

In [ ]:
# modify the instance
# Add 200 to the y coordinate of each point
for p in range(len(LabsFly1.points)):
    LabsFly1.points[p].y = LabsFly1.points[p].y + 200
    
LabsFly1


#ModInst = i + 200 for i in frame_0.instances[0].points

## How to edit all instances

In [ ]:
Insts = frame_0.instances
Insts


In [ ]:

for i in range(len(Insts)):
    Fly = Insts[i]
    for p in range(len(Fly.points)):
        Fly.points[p].y = Fly.points[p].y + 200
        
Insts

# Crop the frames according to arenas

In [ ]:
#im_full_gray = cv2.cvtColor(img_shaped, cv2.COLOR_BGR2GRAY)

plt.figure()
plt.imshow(img_shaped, cmap="gray", vmin=0, vmax=255)

In [ ]:
rows = img_shaped.sum(axis=1)

# convert rows to a list
[rows]

#plt.plot(rows)

In [ ]:
peaks = scipy.signal.find_peaks(rows,
                                distance=40,
                                height=30_000,)

# Check that peaks are correctly located

x = np.array(range(0,len(rows[0])))
PeaksPos = (x[peaks[0]], rows[0][peaks[0]])

plt.plot(rows)
plt.scatter(PeaksPos[0], PeaksPos[1])
plt.show()
#hv.Histogram(rows).opts(tools=['hover'])*hv.Points(PeaksPos).opts(color='orange', tools=['hover'])

In [ ]:
rows[:]

Actually this should be done externally using my already built library. I'll just import cropping parameters to use as transformation values for the new labels.

In [ ]:


NewLabs = sleap.Labels(skeletons=labels.skeletons)



In [ ]:
# Get the path to all the videos in the /Users/ulric/Movies/TrainingVids/Crops folder
PathToCrops = Path("/Users/ulric/Movies/TrainingVids/Crops")
VidPaths = PathToCrops.rglob("*.mp4")
# Put the path to the videos in a list
CropVids = [vid for vid in VidPaths]
CropVids

In [ ]:
labels.videos

In [ ]:
for v in CropVids:
    NewLabs.add_video(v)

NewLabs.videos


In [ ]:
labels.labeled_frames[0]


In [ ]:
#For each frame split instances into separate labels
for f in range(len(labels)):
    frame = labels[f]
    for i in range(len(frame.instances)):
        Fly = frame.instances[i]
        NewLabs.add_instance(Fly, frame_idx=f)

Rotation of labes

In [ ]:
frame0 = labels[0]

In [ ]:
# rotate the points coordinates by 90 degrees
for i in range(len(frame0.instances)):
    Fly = frame0.instances[i]
    for p in range(len(Fly.points)):
        Fly.points[p].x, Fly.points[p].y = Fly.points[p].y, Fly.points[p].x
        